## Import packages

In [ ]:
import os
import pandas as pd 
import numpy as np
import geopandas as gpd  

In [ ]:
import sys
module_path = os.path.abspath('../')
sys.path.append(module_path)
from mobile_capacity.capacity import Capacity
from mobile_capacity.datastorage import DataStorage
from mobile_capacity.entities.pointofinterest import PointOfInterestCollection
from mobile_capacity.entities.cellsite import CellSiteCollection
from mobile_capacity.spatial import plot_layers

In [ ]:
pd.set_option('display.max_columns', None)

## Load input data

In [ ]:
# Get the current directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
data_dir = os.path.join(parent_dir, 'data')
logs_dir = os.path.join(parent_dir, 'logs')
print(f"Check that this is root directory:{parent_dir}")
print(f"Check that this is data directory:{data_dir}")
print(f"Check that this is logs directory:{logs_dir}")

In [ ]:
# Plug in names of datasets in the input folder 
dataset_ids = dict(
    pointofinterest = 'STP-1713385818-4t89-pointofinterest.csv',
    cellsite = 'STP-1713438795-bivd-cellsite.csv')

In [ ]:
# Load data into the data collections
data_storage = DataStorage(data_dir=data_dir)

data_collections = dict(
    pointofinterest = PointOfInterestCollection(),
    cellsite = CellSiteCollection()
)

for data_category in dataset_ids.keys():
    filepath = f"input_data/{dataset_ids[data_category]}"
    entity_records = data_storage.load_data(filepath)
    data_collections[data_category].load_from_records(entity_records)

In [ ]:
# Inspect the data collections
data_collections

In [ ]:
# Read in the optional area file
# area_file = os.path.join(data_dir,'input_data','area.gpkg')
# myarea = gpd.read_file(area_file, driver='GPKG')

## Set analysis parameters

In [ ]:
params = {
        ### Network Configuration ###
        'bw_L850': 5, # MHz on L700 to L900 spectrum bandwidth
        'bw_L1800': 10, # MHz on L1800 spectrum bandwidth
        'bw_L2600': 20, # MHz on L2600 spectrum bandwidth
        'rb_num_multiplier': 5, 
        'cco': 18, # Control channel overheads in %
        'sectors_per_site': 3, # Number of frequency bands on site
        'angles_num': 360, # Set the number of angles to be used for azimuth analysis  
        'rotation_angle': 60, # Define the rotation angle to create a sector +/-rotation_angle degrees clockwise and counter-clockwise  

        ### POI configuration ###
        'dlthtarg': 20, # Download throughput target in Mbps.

        ### Population information ###
        'mbb_subscr': 113, # Active mobile-broadband subscriptions per 100 people, source: https://datahub.itu.int/data/?e=701&c=&i=11632&u=per+100+people
        'oppopshare': 50, # % of Population on Operator
        'dataset_year': 2020, # Year of the population dataset
        'one_km_res': True, # Use 1km resolution population data
        'un_adjusted': True, # Use adjusted population data

        ### Mobile coverage radius ###
        'min_radius': 1000, # meters, minimum radius around cell site location for population calculation
        'max_radius': 2000,  # meters, maximum radius should be divisible by 1000; maximum radius around cell site location for population calculation
        'radius_step': 1000, # meters, radius step size for population calculation

        ### Avg user traffic profile ###
        'nbhours': 10, # number of non-busy hours per day
        'nonbhu': 50, # Connection usage in non-busy hour in % 
    }

## Create an instance of the Capacity class

In [ ]:
# Create an instance of the Capacity class
mobilecapacity = Capacity(country_code="STP",
                          poi=data_collections['pointofinterest'],
                          cellsites=data_collections['cellsite'],
                          data_dir=data_dir,
                          logs_dir=logs_dir,
                          enable_logging=True,
                          **params)

In [ ]:
print(f"The mobile broadband internet traffic per subscription per month in {mobilecapacity.country_code} is: {round(mobilecapacity.udatavmonth_pu,0)} GB per month according to {mobilecapacity.udatavmonth_year} data.")

## Available channel capacity

## Run capacity check for one POI

In [ ]:
# Test: check capacity for some random input values
sufficient_capacity_check = mobilecapacity.capacity_checker(d = 2000, popcd=1000, udatavmonth=mobilecapacity.udatavmonth_pu, pop=5000)
print(f'Capacity is sufficient to connect the POI: {sufficient_capacity_check["capcheck"][0]}.')

## Run capacity check for multiple POIs

In [ ]:
mobilecapacity.capacity_checker(d = [2000,1000,5000], popcd=[800,600,900], udatavmonth=mobilecapacity.udatavmonth_pu, pop=5000)

In [ ]:
mobilecapacity.capacity_checker(d = [2000], popcd=[800,600,900], udatavmonth=mobilecapacity.udatavmonth_pu, pop=5000)

In [ ]:
mobilecapacity.capacity_checker(d = [2000,1000,5000], popcd=[900], udatavmonth=mobilecapacity.udatavmonth_pu, pop=5000)

## Run buffer analysis for all POIs

In [ ]:
# Run buffer analysis
buffer_areas, poi_sufcapch_result = mobilecapacity.calculate_buffer_areas()

In [ ]:
# Buffer areas output data sample
buffer_areas.head(2)

In [ ]:
# POI capacity sufficiency output data sample
poi_sufcapch_result.head(2)

In [ ]:
# Number of True and False POI capacity sufficiency checks
poi_sufcapch_result.sufcapch.value_counts()

## Charts

In [ ]:
# Add map with raster

In [ ]:
# Add map with mobile coverage

In [ ]:
plot_layers(mobilecapacity, poi_sufcapch_result, buffer_areas, show_basemap=True)

In [ ]:
# # Now you can use these boundaries to clip your map
# fig, ax = plt.subplots(figsize=(10, 10))  

# # Display the raster data on the map
# rio_show(raster, ax=ax)  

# # Set the title and axis labels
# ax.set_title('Population density')  
# ax.set_xlabel('Longitude')  
# ax.set_ylabel('Latitude')  

# # Set the limits for the x and y axes using the bounds from the GeoDataFrame
# ax.set_xlim([minx, maxx])
# ax.set_ylim([miny, maxy])

# plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 10))  

# rio_show(raster, ax=ax)  
   
# for radius in range(mobilecapacity.min_radius, mobilecapacity.max_radius+1, mobilecapacity.radius_step):
#         buffer_areas[f'ring_{radius}'].plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
    
# buffer_areas.plot(ax=ax, color='yellow', markersize=30, marker='o', label='Cell site locations', alpha=0.7)

# ax.set_title(f'Ring areas around cell sites for radiuses from {mobilecapacity.min_radius} m to {mobilecapacity.max_radius} m with {mobilecapacity.radius_step} m step.')  
# ax.set_xlabel('Longitude')  
# ax.set_ylabel('Latitude')
# ax.set_xlim([minx, maxx])
# ax.set_ylim([miny, maxy])
# ax.legend() 
 
# plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 10))  
  
# rio_show(raster, ax=ax)  
  
# for radius in range(mobilecapacity.min_radius, mobilecapacity.max_radius+1, mobilecapacity.radius_step):
#         buffer_areas[f'clring_{radius}'].plot(ax=ax, color='blue', edgecolor='lightgrey', alpha=0.5)
  
# buffer_areas.plot(ax=ax, color='yellow', markersize=30, marker='o', label = "Cell site locations", alpha=0.7) 

# ax.set_title('Mobile cellular service coverage areas')  
# ax.set_xlabel('Longitude')  
# ax.set_ylabel('Latitude')
# ax.set_xlim([minx, maxx])
# ax.set_ylim([miny, maxy])
# ax.legend()
 
# plt.show()  

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 10))  

# rio_show(raster, ax=ax)  
  
# for radius in range(mobilecapacity.min_radius, mobilecapacity.max_radius+1, mobilecapacity.radius_step):
#         buffer_areas[f'clring_{radius}'].plot(ax=ax, color='blue', edgecolor='lightgrey', alpha=0.5)
  
# cell_sites.plot(ax=ax, color='yellow', markersize=30, marker='o', label='Cell site locations', alpha=0.7)

# pois[pois['sufcapch'] == True].plot(ax=ax, color='lightgreen',  marker='^', markersize=15, alpha=0.7, label='POI: Sufficient Capacity')
# pois[pois['sufcapch'] == False].plot(ax=ax, color='red',  marker='^', markersize=15, alpha=0.7, label='POI: Insufficient Capacity')

# ax.set_title('Mobile cellular service capacity sufficiency check')
# ax.set_xlabel('Longitude')
# ax.set_ylabel('Latitude')
# ax.set_xlim([minx, maxx])
# ax.set_ylim([miny, maxy])
# ax.legend()

# plt.show()

## Mobile broadband traffic per mobile broadband subscription per month

In [ ]:
# Compile mobile traffic data based on the latest year ITU data available for both
# mobile-broadband Internet traffic (within the country) and active mobile-broadband subscriptions
# and calculates Mobile broadband internet traffic (within the country) per active mobile broadband subscription per month.
mobile_traffic_data = mobilecapacity.mbbtps()

In [ ]:
# Display mobile traffic data sample
mobile_traffic_data.head()

In [ ]:
# Group the data by 'entityIso_mbbsubscr' and calculate the mean traffic per subscriber per month
grouped_data = mobile_traffic_data.loc[mobile_traffic_data.loc[mobile_traffic_data.dataYear == 2022,:].index, ['entityIso_mbbsubscr','mbb_traffic_per_subscr_per_month']].set_index('entityIso_mbbsubscr')

# Define country 
start_position = 30 # the first country to be displayed on the chart (numeration starts with 0)
end_position = 50 # the last country to be displayed on the chart

# Sort the grouped data by the mean traffic values
sorted_data = grouped_data.sort_values(by = 'mbb_traffic_per_subscr_per_month', ascending=False)[start_position:end_position]

fig, ax = plt.subplots(figsize=(10, 10))

sorted_data.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')

ax.set_title('Mean Traffic per Subscriber per Month by Country in 2022')
ax.set_xlabel('Country ISO 3 code')
ax.set_ylabel('Mean Traffic per Subscriber per Month (GB)')
ax.grid(True)

plt.show()

In [ ]:
file_path = os.path.join(mobilecapacity.data_dir, 'output_data', 'MobileBB_Traffic_per_Subscr_per_Month.csv')
if not os.path.exists(file_path):
    mobile_traffic_data.to_csv(file_path)